This notebook is an example how to run two models side-by-side and combine results of both models. A video stream from a local camera is processed by the hand and face detection models. Combined result is then displayed.
OpenCV is required to run this sample.

In [1]:
import degirum as dg # import DeGirum PySDK
import cv2 # OpenCV

In [2]:
# connect to default model zoo
zoo = dg.connect_model_zoo()

In [54]:
# Some helper functions

def show(img, capt = "<image>"):
    # show opencv image
    cv2.imshow(capt, img)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('x') or key == ord('q'):
        raise KeyboardInterrupt
    

In [55]:
# load models for DeGirum Orca AI accelerator
# (change model name to "...n2x_cpu_1" to run it on CPU)
hand_det_model = zoo.load_model("yolo_v5s_hand_det--512x512_quant_n2x_orca_1")
face_det_model = zoo.load_model("yolo_v5s_face_det--512x512_quant_n2x_orca_1")

# select OpenCV backend: needed to have overlay image in OpenCV format
hand_det_model.image_backend = 'opencv'
face_det_model.image_backend = 'opencv' 

In [56]:
# open video stream from local camera #0
stream = cv2.VideoCapture(0)

# define iterator function, which returns frames from camera 
def source(idx):
    # idx is the index of a buffer
    N = 2 # number of buffers
    bufs = [[]] * N
    assert idx < N
    while True:
        if len(bufs[idx]) == 0: # this buffer is empty: get frame from camera and add to all buffers
            ret, frame = stream.read()
            for s in bufs:
                s.insert(0, frame)
        yield bufs[idx].pop()

In [59]:
try:
    # run person detection model on a camera stream
    for hands, faces in zip(hand_det_model.predict_batch(source(0)), face_det_model.predict_batch(source(1))):

        hands._inference_results += faces._inference_results
        show(hands.image_overlay, "Hands and Faces")

except KeyboardInterrupt:
    pass # ignore KeyboardInterrupt errors
finally:
    cv2.destroyAllWindows() # close OpenCV windows


In [52]:
stream.release() # release camera stream